<a href="https://colab.research.google.com/github/HappyJJins/KakaoArena/blob/master/DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive, auth

ROOT = '/content/drive'
drive.mount(ROOT)

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

In [36]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import json
font_path = '/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/NanumGothic.ttf'

import warnings; warnings.simplefilter('ignore')

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 143kB 2.8MB/s 


In [ ]:
genre_gn_all = pd.read_json('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/genre_gn_all.json', typ = 'series')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] == '00']

In [ ]:
song_meta = pd.read_json('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/song_meta.json', typ = 'frame')
train = pd.read_json('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/train.json', typ = 'frame')

In [ ]:
# 곡 아이디(id)와 대분류 장르코드 리스트(song_gn_gnr_basket) 추출
song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]

# unnest song_gn_gnr_basket, 중첩된 songs in basket을 풀어낸다
song_gnr_map_unnest = np.dstack(
    (
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))), 
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values) #배열결합
    )
)

# unnested 데이터프레임 생성 : song_gnr_map
song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns = song_gnr_map.columns)
song_gnr_map['id'] = song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns = {'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace = True)

# unnest 객체 제거
del song_gnr_map_unnest

In [ ]:
# 플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map = train[['id', 'songs']]
# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest

In [ ]:
# 1-1. join 목적으로 곡 아이디 integer to string 변환
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# 1-2. 플레이리스트-곡 테이블 plylst_song_map + 곡-장르 테이블 song_gnr_map join
plylst_song_gnr_map = pd.merge(plylst_song_map, song_gnr_map, how = 'left', left_on = 'songs', right_on = 'song_id')

song_plylst_map = plylst_song_map.sort_values('songs').reset_index()[['songs','id']]

In [ ]:
df = train.loc[:,['id', 'plylst_title', 'tags', 'like_cnt']]
df['id'] = df['id'].astype('str')
df

,id,plylst_title,tags,like_cnt
0,61281,여행같은 음악,[락],71
1,10532,요즘 너 말야,"[추억, 회상]",1
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]",17
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",33
4,27616,추억의 노래 ㅋ,[댄스],9
...,...,...,...,...
115066,120325,METAL E'SM #2,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",3
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,[일렉],13
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",4
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",4


특수 문자 제거

In [ ]:
df['description'] = df['plylst_title'] + ' ' + df['tags'].apply(lambda x: ' '.join(x))
key = '[-=+,#/\?:^$.@*\"~&%!\\|`\(\)\[\]\<\>\']|[ㄱ-ㅎ]'
df['description'] = df['description'].apply(lambda x: re.sub(key,' ',str(x)))
key = '[☆★♡♥♬]'
df['description'] = df['description'].apply(lambda x: re.sub(key, ' ', str(x)))

In [ ]:
df

,id,plylst_title,tags,like_cnt,description
0,61281,여행같은 음악,[락],71,여행같은 음악 락
1,10532,요즘 너 말야,"[추억, 회상]",1,요즘 너 말야 추억 회상
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]",17,편하게 잔잔하게 들을 수 있는 곡 까페 잔잔한
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",33,크리스마스 분위기에 흠뻑 취하고 싶을때 연말 눈오는날 캐럴 분위기 따듯한 크리스마스...
4,27616,추억의 노래 ㅋ,[댄스],9,추억의 노래 댄스
...,...,...,...,...,...
115066,120325,METAL E'SM #2,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",3,METAL E SM 2 록메탈 밴드사운드 록 락메탈 메탈 락 extreme
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,[일렉],13,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음 일렉
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",4,1 눈물이 앞을 가리는 나의_이야기 담시 가족 눈물 그리움 주인공 나의_이야기 ...
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",4,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP 잔잔한 버스 퇴근버스 ...


In [ ]:
des = df['description']
des

0                                                 여행같은 음악 락
1                                             요즘 너 말야 추억 회상
2                              편하게  잔잔하게 들을 수 있는 곡   까페 잔잔한
3         크리스마스 분위기에 흠뻑 취하고 싶을때 연말 눈오는날 캐럴 분위기 따듯한 크리스마스...
4                                               추억의 노래   댄스
                                ...                        
115066           METAL E SM  2 록메탈 밴드사운드 록 락메탈 메탈 락 extreme
115067                    빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음  일렉
115068     1  눈물이 앞을 가리는 나의_이야기 담시 가족 눈물 그리움 주인공 나의_이야기 ...
115069    퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP 잔잔한 버스 퇴근버스 ...
115070                FAVORITE POPSONG    노래추천 팝송추천 팝송 팝송모음
Name: description, Length: 115071, dtype: object

In [ ]:
# des.to_excel('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/des.xlsx', engine='xlsxwriter')

여기서부터는 좀 더 디테일한 전처리리 과정. 추후 공개

In [ ]:
key = re.compile('곡+')
# des.apply(lambda x: key.findall(x))
key.findall(str(des))

['곡']